In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
#from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [137]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [138]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [139]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

#### Решение

Чтобы снизить вычислительную нагрузку, мы выделяем только ТОП-товаров. Остальные товары заменяем на 999999. Опять, же как показали метрики выше, мы не только снижаем вычислительную нагрузку, но и может увеличить точность предсказания. Хотя, как я думаю, это особенность именно этой задачи. В другой задаче, допускаю снижение метрик.

В чатике вы подсказали, что убрать товар 999999 можно с помощью параметра ignore_items у ItemItemRecomender. Не получилось найти это параметр ни с помощью описания методов ? ни в документации в сети.

Я удалил все продажи по товару 999999 просто обнулив количество и сумму продаж. Но, почему, на результирующую метрику это не повлияло никак.

Не пойму, это верный вывод, или я где-то ошибся.

In [140]:
result = pd.read_csv('preds.csv')# загрузка predict с семинара

# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result[a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1019940, 9245287, 961736, 12757364, 848684]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15798461, 132074, 2901256, 3676136, 6554369]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [126]:
#Подготовим данные только топ5000. Удалять не будем, занулим количество, удалять не будем.
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
data_train.loc[~data_train['item_id'].isin(top_5000), 'quantity'] = 0
data_train.loc[~data_train['item_id'].isin(top_5000), 'sales_value'] = 0.

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [127]:
data_train.loc[data_train['item_id'] == 999999].head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
6,2375,26984851516,1,999999,0,0.0,364,-0.68,1642,1,0.0,0.0
8,2375,26984851516,1,999999,0,0.0,364,0.00,1642,1,0.0,0.0


In [128]:
#Сразу опишем пару функций. Пригодятся в дальнейшем

def fit_predict_items_items(column_name, k):
    model = ItemItemRecommender(K=k, num_threads=4, ) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True, )

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)


    result[column_name + str(k)] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])
    
def eval_precision(method: str):
    k = 5
    return result.apply(lambda x: precision_at_k(x[method], x['actual'],  k), axis=1).mean()

In [129]:
fit_predict_items_items('itemitem_top5000_', 5)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_top5000_5
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1019940, 9245287, 961736, 12757364, 848684]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[999999, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15798461, 132074, 2901256, 3676136, 6554369]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[999999, 1082185, 981760, 1098066, 995242]"


In [130]:
prec_itemitem = eval_precision('itemitem') #алгоритм с урока
prec_itemitem_top5000 = eval_precision('itemitem_top5000_5') 

print('precision@5 for itemitem:', '%.3f' % prec_itemitem)
print('precision@5 for itemitem_top5000:', '%.3f' % prec_itemitem_top5000)

precision@5 for itemitem: 0.137
precision@5 for itemitem_top5000: 0.137


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

Сделайте предсказания

#### Решение:
Ниже код самой функции и алгоритма, который готовит массив весов для items

In [141]:
#Загрузим еще раз исходные данные
data = pd.read_csv('data/retail_train.csv')

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [146]:
#Подготовим массив весов
items = data_train.item_id.unique()

sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
weights = sales['sales_value'].apply(lambda x: np.log(x*100) if x > 0 else 0).values

#Уберем в массиве отрицательные значения, почему-то получаются значения, не смотря, что мы ушли от копеек умножив на 100
weights = np.array([x if x >= 0 else 0 for x in weights])

total_sum = weights.sum()
weights = weights / total_sum

print('Проверим, что сумма весов = 1:', weights.sum())
print('Проверим размеры items и weights:', len(items), len(weights))

Проверим, что сумма весов = 1: 1.0
Проверим размеры items и weights: 86865 86865


In [147]:
def weighted_random_recommendation(items, weights, n=5):
    """Взвешенные случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [148]:
items = data_train.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1019940, 9245287, 961736, 12757364, 848684]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[32553, 933079, 12812162, 2757973, 1069614]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15798461, 132074, 2901256, 3676136, 6554369]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[6463442, 937637, 6703891, 6555362, 1028715]"


В решение выше мы рассчитали вес пропорционально логарифму от объема продаж в рублях.

В качестве альтернативы можно было рассчитать весь пропорционально объему продаж (без логарифма).
Или пропорционально количеству проданных товаров (quantity), либо логарифму от проданных товаров
Либо как вариант можно было отсортировать товар в порядке убывания объема продаж (в рублях или количестве) и присвоить весь согласно порядковому номеру. Т.е. сформировать вес зависящий не столько от абсолютного объема продаж, от ранга (позиции) в рейтинге продаж.

Метрику по этому алгоритму в следующем задании посчитаем.

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [149]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1019940, 9245287, 961736, 12757364, 848684]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[32553, 933079, 12812162, 2757973, 1069614]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15798461, 132074, 2901256, 3676136, 6554369]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[6463442, 937637, 6703891, 6555362, 1028715]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1112160, 13417836, 1097280, 7409785, 853309]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[12582220, 1139274, 926908, 17105399, 9446781]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[2010134, 843635, 10150192, 1502922, 2389735]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]","[9297562, 6554774, 6637023, 1064090, 6391101]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[924900, 1048503, 13007479, 274556, 1951532]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[6919202, 1075918, 1070146, 8205098, 1112426]"


#### Решение

In [150]:
prec_random = eval_precision('random_recommendation')
prec_weight_random = eval_precision('weighted_random_recommendation')
prec_popular = eval_precision('popular_recommendation')
prec_itemitem = eval_precision('itemitem')
prec_cosine = eval_precision('cosine')
prec_tfidf = eval_precision('tfidf')
prec_own_purchases = eval_precision('own_purchases')


print('precision@5 for random_recommendation:', '%.3f' % prec_random)
print('precision@5 for weighted_random_recommendation:', '%.3f' % prec_weight_random)
print('precision@5 for popular_recommendation:', '%.3f' % prec_popular)
print('precision@5 for itemitem:', '%.3f' % prec_itemitem)
print('precision@5 for cosine:', '%.3f' % prec_cosine)
print('precision@5 for tfidf:', '%.3f' % prec_tfidf)
print('precision@5 for own_purchases:', '%.3f' % prec_own_purchases)

precision@5 for random_recommendation: 0.000
precision@5 for weighted_random_recommendation: 0.000
precision@5 for popular_recommendation: 0.155
precision@5 for itemitem: 0.137
precision@5 for cosine: 0.133
precision@5 for tfidf: 0.139
precision@5 for own_purchases: 0.180


Рандомные рекомендации и взвешенные рандомные рекомендации показали самое ужасное качество, на уровне 0.
Моё мнение использовать их в качестве бейзлайна неприемлемо. Какой смысл сравниваться с нулём.

Что оказалось для меня удивительным, математические сложные и "умные" алгоритмы вроде косинусного расстояния и tfidf показали худший результат, чем банальный ТОП-5 одинаковых рекомендаций для всех. 

И под конец приятно удивил результат рекомендаций на основе собственных покупок, показавший максимальный результат. По-моему мнению, рекомендации на основе исключительно собственных покупок получились самые персонализированные. Возможно мы имеем дело с датасетом, где большая доля именно повторных покупок. Возможно поэтому персонализированные рекомендации на основе собственных покупок сработали лучше всего.

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


#### Решение

In [151]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [152]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [153]:
#Подготовим массив весов top_5000
sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()[:5000]
weights = sales['sales_value'].apply(lambda x: np.log(x*100) if x > 0 else 0).values

#Уберем в массиве отрицательные значения, почему-то получаются значения, не смотря, что мы ушли от копеек умножив на 100
weights = np.array([x if x >= 0 else 0 for x in weights])

total_sum = weights.sum()
weights = weights / total_sum

print('Проверим, что сумма весов = 1:', weights.sum())
print('Проверим размеры top_5000 и weights:', len(top_5000), len(weights))

Проверим, что сумма весов = 1: 0.9999999999999999
Проверим размеры top_5000 и weights: 5000 5000


In [154]:
#посчитаем бейзлайны на топ-5000
result['random_rec_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result['weighted_random_rec_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(top_5000, weights, n=5))

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_rec_5000,weighted_random_rec_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1019940, 9245287, 961736, 12757364, 848684]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[32553, 933079, 12812162, 2757973, 1069614]","[9837692, 843316, 14024954, 1006440, 962850]","[1055168, 977854, 971195, 1115801, 9858748]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15798461, 132074, 2901256, 3676136, 6554369]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[6463442, 937637, 6703891, 6555362, 1028715]","[8068578, 12262983, 6753831, 6463721, 8119159]","[869024, 999581, 998558, 5995692, 886803]"


In [155]:
#Подготовим данные для items-items. Ниже мы повторим его еще несколько раз

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [157]:
fit_predict_items_items('itemitem_', 2)
fit_predict_items_items('itemitem_', 10)
fit_predict_items_items('itemitem_', 20)

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_rec_5000,weighted_random_rec_5000,itemitem_2,itemitem_10,itemitem_20
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1019940, 9245287, 961736, 12757364, 848684]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[32553, 933079, 12812162, 2757973, 1069614]","[9837692, 843316, 14024954, 1006440, 962850]","[1055168, 977854, 971195, 1115801, 9858748]","[999999, 1082185, 995242, 1029743, 840361]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15798461, 132074, 2901256, 3676136, 6554369]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[6463442, 937637, 6703891, 6555362, 1028715]","[8068578, 12262983, 6753831, 6463721, 8119159]","[869024, 999581, 998558, 5995692, 886803]","[999999, 1082185, 1098066, 6534178, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]"


In [158]:
prec_random = eval_precision('random_rec_5000')
prec_weight_random = eval_precision('weighted_random_rec_5000')
prec_itemitem2 = eval_precision('itemitem_2')
prec_itemitem10 = eval_precision('itemitem_10')
prec_itemitem20 = eval_precision('itemitem_20')

print('precision@5 for random_rec_5000:', '%.3f' % prec_random)
print('precision@5 for weighted_random_rec_5000:', '%.3f' % prec_weight_random)
print('precision@5 for itemitem_2:', '%.3f' % prec_itemitem2)
print('precision@5 for itemitem_10:', '%.3f' % prec_itemitem10)
print('precision@5 for itemitem_20:', '%.3f' % prec_itemitem20)

precision@5 for random_rec_5000: 0.001
precision@5 for weighted_random_rec_5000: 0.007
precision@5 for itemitem_2: 0.192
precision@5 for itemitem_10: 0.151
precision@5 for itemitem_20: 0.153


Рандомные и взвешенные рандомные рекомендации на топ-5000 улучшились. Что и логично. Рандом стал выбираться из меньшего числа элементов.

Алгоритм item-item с другим количеством соседей, тоже показал более хороший результат. Например, при количестве соседей К=2, результат оказался даже лучше, чем при К=1. Его значение 0.192

Зато при повышении К от 5 (базовое значение) результат рос, но где где-то после К=10 остановился на одном значении около 0.151.

Отсюда сделаем вывод, что параметр количество соседей надо подбирать. И нельзя однозначно сказать, что чем больше или чем меньше, тем лучше.

Ансамблирование сделать не смог, так как не пойму, как ансамблировать эти типы алгоритмов. Буду признателен, если мы разберем пример ансамблирования на уроке.

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

#### не сделал

не смог сообразить как. Очень интересно было бы разбор увидеть потом этого задания.